# Hands on HDF5
This brief lab session gives an overview over the basic usage of the HDF5 file format for Python applications.

Work through the examples and try to change some of the settings...

### Sources:
* HDF5 documentation https://portal.hdfgroup.org/display/HDF5/HDF5
* h5py API: http://docs.h5py.org/en/stable/



## Creating a Data Set

In [ ]:
import numpy as np
import h5py #this is the HDF5 lib 

In [ ]:
#create some random data
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (10000,100))

In [ ]:
# write it to the same file - in two different arrays
with h5py.File('hdf5_data.h5', 'w') as hdf: #note the write mode 'w'
    hdf.create_dataset('dataset1', data=matrix1)
    hdf.create_dataset('dataset2', data=matrix2)

## Reading 

In [ ]:
#opening, listing and reading files
with h5py.File('hdf5_data.h5','r') as hdf:
    ls = list(hdf.keys())
    print('List of datasets in this file: \n', ls)
    data = hdf.get('dataset2') #here data is still some hdf5 object
    dataset1 = np.array(data) #need to convert it into numpy
    print('Shape of dataset1: \n', dataset1.shape)

In [ ]:
dataset1

In [ ]:
f = h5py.File('hdf5_data.h5', 'r')
ls = list(f.keys())
f.close()

In [ ]:
ls

## Array Slicing
HDF5 support fancy array slicing - so we do not read all data just to get a slice: http://docs.h5py.org/en/latest/high/dataset.html#fancy-indexing

In [ ]:
f = h5py.File('hdf5_data.h5', 'r')
f['dataset1'][100:120,:] # this notation mostly follows numpy notation -> try different slices!

## Creating Groups
We can organize data in groups, just like in file systems where we have files (here datasets) in folders (here groups) 

In [ ]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (1000,1000))
matrix3 = np.random.random(size = (1000,1000))
matrix4 = np.random.random(size = (1000,1000))

In [ ]:
with h5py.File('hdf5_groups.h5', 'w') as hdf:
    G1 = hdf.create_group('Group1')
    G1.create_dataset('dataset1', data = matrix1)
    G1.create_dataset('dataset4', data = matrix4)
 
    G21 = hdf.create_group('Group2/SubGroup1')
    G21.create_dataset('dataset3', data = matrix3)
    
    G22 = hdf.create_group('Group2/SubGroup2')
    G22.create_dataset('dataset2', data = matrix2)

## Reading Groups

In [ ]:
with h5py.File('hdf5_groups.h5','r') as hdf:
    base_items = list(hdf.items())
    print('Items in the base directory:', base_items)
    G2 = hdf.get('Group2')
    G2_items = list(G2.items())
    print('Items in Group2:', G2_items)
    G21 = G2.get('/Group2/SubGroup1')
    G21_items = list(G21.items())
    print('Items in Group21:', G21_items)
    dataset3 = np.array(G21.get('dataset3'))
    print(dataset3.shape)


### What is happening? Interpret the results.

## Compress Data
HDF5 also support native data compression:

In [ ]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (1000,1000))
matrix3 = np.random.random(size = (1000,1000))
matrix4 = np.random.random(size = (1000,1000))

In [ ]:
with h5py.File('hdf5_groups_compressed.h5', 'w') as hdf:
    G1 = hdf.create_group('Group1')
    G1.create_dataset('dataset1', data = matrix1, compression="gzip", compression_opts=9)
    G1.create_dataset('dataset4', data = matrix4, compression="gzip", compression_opts=9)
 
    G21 = hdf.create_group('Group2/SubGroup1')
    G21.create_dataset('dataset3', data = matrix3, compression="gzip", compression_opts=9)
    
    G22 = hdf.create_group('Group2/SubGroup2')
    G22.create_dataset('dataset2', data = matrix2, compression="gzip", compression_opts=9)

## Attributes
We can add meta information in form of attributes of files, groups and datasets:

In [ ]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (10000,100))

In [ ]:
# Create the HDF5 file
hdf = h5py.File('test.h5', 'w')

# Create the datasets
dataset1 = hdf.create_dataset('dataset1', data=matrix1)
dataset2 = hdf.create_dataset('dataset2', data=matrix2)

# Set attributes
dataset1.attrs['CLASS'] = 'DATA MATRIX'
dataset1.attrs['VERSION'] = '1.1'

hdf.close()

In [ ]:
# Read the HDF5 file
hdf = h5py.File('test.h5', 'r')
ls = list(hdf.keys())
print('List of datasets in this file: \n', ls)
data = hdf.get('dataset1')
dataset1 = np.array(data)
print('Shape of dataset1: \n', dataset1.shape)
#read the attributes
k = list(data.attrs.keys())
v = list(data.attrs.values())
print(k[0])
print(v[0])
print(data.attrs[k[0]])

hdf.close()

## HDF5 and ***Pandas***

In [ ]:
import pandas as pd
# creates (or opens in append mode) an hdf5 file
hdf = pd.HDFStore('hdf5_pandas.h5')

In [ ]:
df1 = pd.read_csv('../../DATA/FL_insurance_sample.csv')# put the dataset in the storage
hdf.put('DF1', df1, format='table', data_columns=True)

In [ ]:
data = {
         "city": ["Tripoli", "Sydney", "Tripoli", "Rome", "Rome", "Tripoli","Rome", "Sydney", "Sydney"],
         "rank": ["1st", "2nd", "1st", "2nd", "1st", "2nd","1st", "2nd", "1st"], 
         "score1": [44, 48, 39, 41, 38, 44, 34, 54, 61],
         "score2": [67, 63, 55, 70, 64, 77, 45, 66, 72]
        }
        
df2 = pd.DataFrame(data, columns = ['city', 'rank','score1','score2'])

In [ ]:
df2

In [ ]:
hdf.put('DF2Key', df2,format='table', data_columns=True)

In [ ]:
hdf.close() # close the hdf5 file

### now read

In [ ]:
# open hdf5 file for reading
hdf = pd.HDFStore('hdf5_pandas.h5',mode='r')

In [ ]:
hdf.keys()

In [ ]:
df1 = hdf.get('/DF1')

In [ ]:
type(df1)

In [ ]:
df1.head()

In [ ]:
hdf.close()